### Loading Packages and Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

#import xarray as xr
import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
from db_queries import get_population

import altair as alt
from vega_datasets import data

In [2]:
ls /mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/19.5_51_locations/results_data

binned_ldl_exposure_time.csv     risk_exposure_time.csv
binned_ldl_exposure_time.hdf     risk_exposure_time.hdf
binned_sbp_exposure_time.csv     sbp_medication_person_time.csv
binned_sbp_exposure_time.hdf     sbp_medication_person_time.hdf
deaths.csv                       state_person_time.csv
deaths.hdf                       state_person_time.hdf
healthcare_visits.csv            transition_count.csv
healthcare_visits.hdf            transition_count.hdf
intervention_person_time.csv     ylds.csv
intervention_person_time.hdf     ylds.hdf
ldlc_medication_person_time.csv  ylls.csv
ldlc_medication_person_time.hdf  ylls.hdf


In [3]:
sim_results_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/19.5_51_locations/results_data/'

# Loading and Cleaning Data

### Load Data

In [44]:
tran = pd.read_csv(sim_results_dir + 'transition_count.csv')

state_person_time = pd.read_csv(sim_results_dir + '/state_person_time.csv')
person_time = state_person_time.loc[state_person_time['state'].isin(["acute_ischemic_stroke", "chronic_ischemic_stroke", "susceptible_to_ischemic_stroke"])].reset_index() 

In [5]:
yll = pd.read_csv(sim_results_dir + 'ylls.csv').reset_index()
yll = yll.rename(columns = {'value':'yll'})
yld = pd.read_csv(sim_results_dir + 'ylds.csv').reset_index()
yld = yld.rename(columns = {'value':'yld', 'cause_of_disability':'cause'})
daly = yll.merge(yld, on = ['sex','year','cause','input_draw','scenario','age','location'], how='left')
daly['daly'] = daly.yll.fillna(0) + daly.yld.fillna(0)
daly = daly[['sex','year','cause','input_draw','scenario','age','location','yll','yld','daly']]
daly.head()

,sex,year,cause,input_draw,scenario,age,location,yll,yld,daly
0,Female,2021,acute_ischemic_stroke,829,baseline,25_to_29,washington,0.0,0.000088,0.000088
1,Female,2021,acute_ischemic_stroke,829,lifestyle_100,25_to_29,washington,0.0,0.000088,0.000088
2,Female,2021,acute_ischemic_stroke,829,outreach_100,25_to_29,washington,0.0,0.000088,0.000088
3,Female,2021,acute_ischemic_stroke,829,polypill_100,25_to_29,washington,0.0,0.000088,0.000088
4,Female,2021,acute_ischemic_stroke,829,baseline,30_to_34,washington,0.0,0.000000,0.000000


### Load Population Data for Scaling 

In [6]:
US_pop = get_population(age_group_id=157, 
                              location_id=[102,523, 524, 525, 526, 527, 528, 529, 530,531, 532, 533,534, 535,
                                           536, 537, 538, 539, 540,541, 542, 543, 544 ,545, 546, 547, 548, 549, 550,
                                           551, 552, 553, 554, 555, 556, 557, 558, 559, 560,561, 562, 563, 564, 565, 
                                           566, 567, 568, 569, 570,571, 572, 573], 
                              year_id=2019, 
                              sex_id=3, 
                              release_id=9)
US_pop.head()

,age_group_id,location_id,year_id,sex_id,population,run_id
0,157,102,2019,3,2.260218e+08,359
1,157,523,2019,3,3.425549e+06,359
2,157,524,2019,3,4.848432e+05,359
3,157,525,2019,3,4.812495e+06,359
4,157,526,2019,3,2.020596e+06,359


In [7]:
location_map = pd.read_csv('/ihme/homes/lutzes/vivarium_research_nih_us_cvd/CVD_locations.csv')
US_pop = US_pop.merge(location_map,on=['location_id'])
US_pop['scalar'] = US_pop['population'] / 50_000
US_pop

,age_group_id,location_id,year_id,sex_id,population,run_id,location_name,abbreviation,scalar
0,157,102,2019,3,2.260218e+08,359,US,US,4520.435794
1,157,523,2019,3,3.425549e+06,359,Alabama,AL,68.510979
2,157,524,2019,3,4.848432e+05,359,Alaska,AK,9.696864
3,157,525,2019,3,4.812495e+06,359,Arizona,AZ,96.249902
...,...,...,...,...,...,...,...,...,...
48,157,570,2019,3,5.265596e+06,359,Washington,WA,105.311924
49,157,571,2019,3,1.283409e+06,359,West Virginia,WV,25.668176
50,157,572,2019,3,4.056749e+06,359,Wisconsin,WI,81.134990
51,157,573,2019,3,3.913808e+05,359,Wyoming,WY,7.827615


### Scale Datasets

# Create Dataset with Needed Outcomes 
•	MI, stroke, and HF incidence rates + counts 

•	IHD (MI + post-MI + HF from IHD), stroke, and HF DALYs 

•	Percent change and counts 

•	National and PDFs with states 

### Starting with Incidence Counts for MI, stroke, and HF 

In [45]:
person_time = person_time.groupby(['input_draw','year','scenario','location']).value.sum().reset_index()
person_time_allyears = person_time.groupby(['input_draw','scenario','location']).value.sum().reset_index()
person_time_allyears = person_time_allyears.rename(columns = {'value':'ptvalue'})
person_time = person_time.rename(columns = {'value':'ptvalue'})
#person_time

In [9]:
def q25(x):
    return x.quantile(0.025)

def q975(x):
    return x.quantile(0.975)

def convert(word):
    return ' '.join(x.capitalize() or '_' for x in word.split('_'))

In [10]:
convert('new_hampshire')

'New Hampshire'

In [11]:
tran['transition_new'] = np.where(tran.transition.isin(['susceptible_to_ischemic_heart_disease_and_heart_failure_to_acute_myocardial_infarction','heart_failure_from_ischemic_heart_disease_to_acute_myocardial_infarction_and_heart_failure','post_myocardial_infarction_to_acute_myocardial_infarction']), 'Myocardial Infarction', np.where(tran.transition.isin(['susceptible_to_ischemic_stroke_to_acute_ischemic_stroke','chronic_ischemic_stroke_to_acute_ischemic_stroke']),'Ischemic Stroke','Heart Failure'))
tran_state_rate = tran.groupby(['input_draw','year','scenario','location','transition_new']).value.sum().reset_index()
tran_state_rate['location_name'] = tran_state_rate.location.apply(convert)
tran_state_rate = tran_state_rate.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time[['input_draw','year','scenario','location','ptvalue']], on=['input_draw','year','scenario','location'])
tran_state_rate['rate'] = tran_state_rate['value'] / tran_state_rate['ptvalue']
#tran_state_rate

In [46]:
tran['transition_new'] = np.where(tran.transition.isin(['susceptible_to_ischemic_heart_disease_and_heart_failure_to_acute_myocardial_infarction','heart_failure_from_ischemic_heart_disease_to_acute_myocardial_infarction_and_heart_failure','post_myocardial_infarction_to_acute_myocardial_infarction']), 'Myocardial Infarction', np.where(tran.transition.isin(['susceptible_to_ischemic_stroke_to_acute_ischemic_stroke','chronic_ischemic_stroke_to_acute_ischemic_stroke']),'Ischemic Stroke','Heart Failure'))
tran_state_rate_allyear = tran.groupby(['input_draw','scenario','location','transition_new']).value.sum().reset_index()
tran_state_rate_allyear['location_name'] = tran_state_rate_allyear.location.apply(convert)
tran_state_rate_allyear = tran_state_rate_allyear.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time_allyears[['input_draw','scenario','location','ptvalue']], on=['input_draw','scenario','location'])
tran_state_rate_allyear['rate'] = tran_state_rate_allyear['value'] / tran_state_rate_allyear['ptvalue']
tran_state_rate_allyear.head()

,input_draw,scenario,location,transition_new,value,location_name,abbreviation,scalar,ptvalue,rate
0,829,baseline,alabama,Heart Failure,11726.0,Alabama,AL,68.510979,719947.192334,0.016287
1,829,baseline,alabama,Ischemic Stroke,1250.0,Alabama,AL,68.510979,719947.192334,0.001736
2,829,baseline,alabama,Myocardial Infarction,3603.0,Alabama,AL,68.510979,719947.192334,0.005005
3,829,lifestyle_100,alabama,Heart Failure,11667.0,Alabama,AL,68.510979,720205.459274,0.016200
4,829,lifestyle_100,alabama,Ischemic Stroke,1241.0,Alabama,AL,68.510979,720205.459274,0.001723


In [47]:
tran_state_rate_allyear_baseline = tran_state_rate_allyear.loc[tran_state_rate_allyear.scenario == 'baseline']
tran_state_rate_allyear_baseline = tran_state_rate_allyear_baseline.rename(columns = {'rate':'baseline'})
tran_state_rate_allyear_total = tran_state_rate_allyear.merge(tran_state_rate_allyear_baseline[['input_draw','location_name','transition_new','baseline']], on=['input_draw','location_name','transition_new']) 
tran_state_rate_allyear_total['averted_count'] = tran_state_rate_allyear_total['baseline'] - tran_state_rate_allyear_total['rate']
tran_state_rate_allyear_total['averted_percent'] = 100*((tran_state_rate_allyear_total['baseline'] - tran_state_rate_allyear_total['rate'])/tran_state_rate_allyear_total['baseline'])
tran_state_rate_allyear_total.head()

,input_draw,scenario,location,transition_new,value,location_name,abbreviation,scalar,ptvalue,rate,baseline,averted_count,averted_percent
0,829,baseline,alabama,Heart Failure,11726.0,Alabama,AL,68.510979,719947.192334,0.016287,0.016287,0.000000,0.000000
1,829,lifestyle_100,alabama,Heart Failure,11667.0,Alabama,AL,68.510979,720205.459274,0.016200,0.016287,0.000088,0.538835
2,829,outreach_100,alabama,Heart Failure,11682.0,Alabama,AL,68.510979,720210.212183,0.016220,0.016287,0.000067,0.411617
3,829,polypill_100,alabama,Heart Failure,11357.0,Alabama,AL,68.510979,721479.775496,0.015741,0.016287,0.000546,3.352591
4,829,baseline,alabama,Ischemic Stroke,1250.0,Alabama,AL,68.510979,719947.192334,0.001736,0.001736,0.000000,0.000000


In [48]:
tran_state_rate_baseline = tran_state_rate.loc[tran_state_rate.scenario == 'baseline']
tran_state_rate_baseline = tran_state_rate_baseline.rename(columns = {'rate':'baseline'})
tran_state_rate_total = tran_state_rate.merge(tran_state_rate_baseline[['input_draw','year','location_name','transition_new','baseline']], on=['input_draw','year','location_name','transition_new']) 
tran_state_rate_total['averted_count'] = tran_state_rate_total['baseline'] - tran_state_rate_total['rate']
tran_state_rate_total['averted_percent'] = 100*((tran_state_rate_total['baseline'] - tran_state_rate_total['rate'])/tran_state_rate_total['baseline'])
#tran_state_rate_total.head()

In [49]:
tran_state_rate_summ = (tran_state_rate_total
                      .groupby(['year','location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
#tran_state_rate_summ.head()

In [51]:
tran_state_rate_allyear_summ = (tran_state_rate_allyear_total
                      .groupby(['location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
tran_state_rate_allyear_summ.head()

averted_count  \
                                                                  mean   
location_name abbreviation transition_new  scenario                      
Alabama       AL           Heart Failure   baseline           0.000000   
                                           lifestyle_100      0.000088   
                                           outreach_100       0.000067   
                                           polypill_100       0.000546   
                           Ischemic Stroke baseline           0.000000   

                                                                              \
                                                               q25      q975   
location_name abbreviation transition_new  scenario                            
Alabama       AL           Heart Failure   baseline       0.000000  0.000000   
                                           lifestyle_100  0.000088  0.000088   
                                           outreach_100   0.000067  0.000067   
                                           polypill_100   0.000546  0.000546   
                           Ischemic Stroke baseline       0.000000  0.000000   

                                                         averted_percent  \
                                                                    mean   
location_name abbreviation transition_new  scenario                        
Alabama       AL           Heart Failure   baseline             0.000000   
                                           lifestyle_100        0.538835   
                                           outreach_100         0.411617   
                                           polypill_100         3.352591   
                           Ischemic Stroke baseline             0.000000   

                                                                              \
                                                               q25      q975   
location_name abbreviation transition_new  scenario                            
Alabama       AL           Heart Failure   baseline       0.000000  0.000000   
                                           lifestyle_100  0.538835  0.538835   
                                           outreach_100   0.411617  0.411617   
                                           polypill_100   3.352591  3.352591   
                           Ischemic Stroke baseline       0.000000  0.000000   

                                                              rate            \
                                                              mean       q25   
location_name abbreviation transition_new  scenario                            
Alabama       AL           Heart Failure   baseline       0.016287  0.016287   
                                           lifestyle_100  0.016200  0.016200   
                                           outreach_100   0.016220  0.016220   
                                           polypill_100   0.015741  0.015741   
                           Ischemic Stroke baseline       0.001736  0.001736   

                                                                    
                                                              q975  
location_name abbreviation transition_new  scenario                 
Alabama       AL           Heart Failure   baseline       0.016287  
                                           lifestyle_100  0.016200  
                                           outreach_100   0.016220  
                                           polypill_100   0.015741  
                           Ischemic Stroke baseline       0.001736

In [14]:
tran_state_rate_summ.columns = ["_".join(a) for a in tran_state_rate_summ.columns.to_flat_index()]
tran_state_rate_summ.reset_index(inplace=True)
tran_state_rate_summ.loc[(tran_state_rate_summ.scenario == 'baseline') & (tran_state_rate_summ.transition_new == 'Heart Failure')]
#tran_state_rate_summ.head()

,year,location_name,abbreviation,transition_new,scenario,averted_count_mean,averted_count_q25,averted_count_q975,averted_percent_mean,averted_percent_q25,averted_percent_q975,rate_mean,rate_q25,rate_q975
0,2021,Alabama,AL,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.016511,0.016511,0.016511
12,2021,Alaska,AK,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.010406,0.010406,0.010406
24,2021,Arizona,AZ,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.015103,0.015103,0.015103
36,2021,Arkansas,AR,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.016907,0.016907,0.016907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11952,2040,Washington,WA,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.015596,0.015596,0.015596
11964,2040,West Virginia,WV,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.020252,0.020252,0.020252
11976,2040,Wisconsin,WI,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.017680,0.017680,0.017680
11988,2040,Wyoming,WY,Heart Failure,baseline,0.0,0.0,0.0,0.0,0.0,0.0,0.016282,0.016282,0.016282


In [52]:
tran_state_rate_allyear_summ.columns = ["_".join(a) for a in tran_state_rate_allyear_summ.columns.to_flat_index()]
tran_state_rate_allyear_summ.reset_index(inplace=True)
tran_state_rate_allyear_summ.head()

,location_name,abbreviation,transition_new,scenario,averted_count_mean,averted_count_q25,averted_count_q975,averted_percent_mean,averted_percent_q25,averted_percent_q975,rate_mean,rate_q25,rate_q975
0,Alabama,AL,Heart Failure,baseline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016287,0.016287,0.016287
1,Alabama,AL,Heart Failure,lifestyle_100,0.000088,0.000088,0.000088,0.538835,0.538835,0.538835,0.016200,0.016200,0.016200
2,Alabama,AL,Heart Failure,outreach_100,0.000067,0.000067,0.000067,0.411617,0.411617,0.411617,0.016220,0.016220,0.016220
3,Alabama,AL,Heart Failure,polypill_100,0.000546,0.000546,0.000546,3.352591,3.352591,3.352591,0.015741,0.015741,0.015741
4,Alabama,AL,Ischemic Stroke,baseline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001736,0.001736,0.001736


### Now Making Datasets for DALYs for IHD, stroke, and heart failure

In [15]:
daly = daly.loc[daly.cause!='other_causes']
daly['transition_new'] = np.where(daly.cause.isin(['heart_failure_residual']), 'Heart Failure', np.where(daly.cause.isin(['acute_ischemic_stroke','chronic_ischemic_stroke']),'Ischemic Stroke','Ischemic Heart Disease'))
daly_state = daly.groupby(['input_draw','year','scenario','location','transition_new']).daly.sum().reset_index()
daly_state['location_name'] = daly_state.location.apply(convert)
daly_state_rate = daly_state.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time[['input_draw','year','scenario','location','ptvalue']], on=['input_draw','year','scenario','location'])
daly_state_rate['rate'] = daly_state_rate['daly'] / daly_state_rate['ptvalue']
#daly_state_rate

In [60]:
daly = daly.loc[daly.cause!='other_causes']
daly['transition_new'] = np.where(daly.cause.isin(['heart_failure_residual']), 'Heart Failure', np.where(daly.cause.isin(['acute_ischemic_stroke','chronic_ischemic_stroke']),'Ischemic Stroke','Ischemic Heart Disease'))
daly_state_allyear = daly.groupby(['input_draw','scenario','location','transition_new']).daly.sum().reset_index()
daly_state_allyear['location_name'] = daly_state_allyear.location.apply(convert)
daly_state_allyear_rate = daly_state_allyear.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time_allyears[['input_draw','scenario','location','ptvalue']], on=['input_draw','scenario','location'])
daly_state_allyear_rate['rate'] = daly_state_allyear_rate['daly'] / daly_state_allyear_rate['ptvalue']
daly_state_allyear_rate.head()

,input_draw,scenario,location,transition_new,daly,location_name,abbreviation,scalar,ptvalue,rate
0,829,baseline,alabama,Heart Failure,62216.649630,Alabama,AL,68.510979,719947.192334,0.086418
1,829,baseline,alabama,Ischemic Heart Disease,93554.422577,Alabama,AL,68.510979,719947.192334,0.129946
2,829,baseline,alabama,Ischemic Stroke,10032.259125,Alabama,AL,68.510979,719947.192334,0.013935
3,829,lifestyle_100,alabama,Heart Failure,62207.615969,Alabama,AL,68.510979,720205.459274,0.086375
4,829,lifestyle_100,alabama,Ischemic Heart Disease,93010.301948,Alabama,AL,68.510979,720205.459274,0.129144


In [16]:
daly_state_rate_baseline = daly_state_rate.loc[daly_state_rate.scenario == 'baseline']
daly_state_rate_baseline = daly_state_rate_baseline.rename(columns = {'rate':'baseline'})
daly_state_rate_total = daly_state_rate.merge(daly_state_rate_baseline[['input_draw','year','location_name','transition_new','baseline']], on=['input_draw','year','location_name','transition_new']) 
daly_state_rate_total['averted_count'] = daly_state_rate_total['baseline'] - daly_state_rate_total['rate']
daly_state_rate_total['averted_percent'] = 100*((daly_state_rate_total['baseline'] - daly_state_rate_total['rate'])/daly_state_rate_total['baseline'])
#daly_state_rate_total#.head()

In [61]:
daly_state_allyear_rate_baseline = daly_state_allyear_rate.loc[daly_state_allyear_rate.scenario == 'baseline']
daly_state_allyear_rate_baseline = daly_state_allyear_rate_baseline.rename(columns = {'rate':'baseline'})
daly_state_allyear_rate_total = daly_state_allyear_rate.merge(daly_state_allyear_rate_baseline[['input_draw','location_name','transition_new','baseline']], on=['input_draw','location_name','transition_new']) 
daly_state_allyear_rate_total['averted_count'] = daly_state_allyear_rate_total['baseline'] - daly_state_allyear_rate_total['rate']
daly_state_allyear_rate_total['averted_percent'] = 100*((daly_state_allyear_rate_total['baseline'] - daly_state_allyear_rate_total['rate'])/daly_state_allyear_rate_total['baseline'])
daly_state_allyear_rate_total.head()

,input_draw,scenario,location,transition_new,daly,location_name,abbreviation,scalar,ptvalue,rate,baseline,averted_count,averted_percent
0,829,baseline,alabama,Heart Failure,62216.649630,Alabama,AL,68.510979,719947.192334,0.086418,0.086418,0.000000,0.000000
1,829,lifestyle_100,alabama,Heart Failure,62207.615969,Alabama,AL,68.510979,720205.459274,0.086375,0.086418,0.000044,0.050375
2,829,outreach_100,alabama,Heart Failure,62095.710096,Alabama,AL,68.510979,720210.212183,0.086219,0.086418,0.000199,0.230833
3,829,polypill_100,alabama,Heart Failure,61352.150015,Alabama,AL,68.510979,721479.775496,0.085037,0.086418,0.001382,1.598969
4,829,baseline,alabama,Ischemic Heart Disease,93554.422577,Alabama,AL,68.510979,719947.192334,0.129946,0.129946,0.000000,0.000000


In [67]:
daly_state_allyear_rate_summ = (daly_state_allyear_rate_total
                      .groupby(['location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
daly_state_allyear_rate_summ.head()

averted_count  \
                                                                         mean   
location_name abbreviation transition_new         scenario                      
Alabama       AL           Heart Failure          baseline           0.000000   
                                                  lifestyle_100      0.000044   
                                                  outreach_100       0.000199   
                                                  polypill_100       0.001382   
                           Ischemic Heart Disease baseline           0.000000   

                                                                           \
                                                                      q25   
location_name abbreviation transition_new         scenario                  
Alabama       AL           Heart Failure          baseline       0.000000   
                                                  lifestyle_100  0.000044   
                                                  outreach_100   0.000199   
                                                  polypill_100   0.001382   
                           Ischemic Heart Disease baseline       0.000000   

                                                                           \
                                                                     q975   
location_name abbreviation transition_new         scenario                  
Alabama       AL           Heart Failure          baseline       0.000000   
                                                  lifestyle_100  0.000044   
                                                  outreach_100   0.000199   
                                                  polypill_100   0.001382   
                           Ischemic Heart Disease baseline       0.000000   

                                                                averted_percent  \
                                                                           mean   
location_name abbreviation transition_new         scenario                        
Alabama       AL           Heart Failure          baseline             0.000000   
                                                  lifestyle_100        0.050375   
                                                  outreach_100         0.230833   
                                                  polypill_100         1.598969   
                           Ischemic Heart Disease baseline             0.000000   

                                                                           \
                                                                      q25   
location_name abbreviation transition_new         scenario                  
Alabama       AL           Heart Failure          baseline       0.000000   
                                                  lifestyle_100  0.050375   
                                                  outreach_100   0.230833   
                                                  polypill_100   1.598969   
                           Ischemic Heart Disease baseline       0.000000   

                                                                           \
                                                                     q975   
location_name abbreviation transition_new         scenario                  
Alabama       AL           Heart Failure          baseline       0.000000   
                                                  lifestyle_100  0.050375   
                                                  outreach_100   0.230833   
                                                  polypill_100   1.598969   
                           Ischemic Heart Disease baseline       0.000000   

                                                                     rate  \
                                                                     mean   
location_name abbreviation transition_new         scenario                  
Alabama       AL           Heart Failure          b

In [ ]:
daly_state_rate_summ = (daly_state_rate_total
                      .groupby(['year','location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
#daly_state_rate_summ.head()

In [18]:
daly_state_rate_summ.columns = ["_".join(a) for a in daly_state_rate_summ.columns.to_flat_index()]
daly_state_rate_summ.reset_index(inplace=True)
#daly_state_rate_summ.head()

In [68]:
daly_state_allyear_rate_summ.columns = ["_".join(a) for a in daly_state_allyear_rate_summ.columns.to_flat_index()]
daly_state_allyear_rate_summ.reset_index(inplace=True)
daly_state_allyear_rate_summ.head()

,location_name,abbreviation,transition_new,scenario,averted_count_mean,averted_count_q25,averted_count_q975,averted_percent_mean,averted_percent_q25,averted_percent_q975,rate_mean,rate_q25,rate_q975
0,Alabama,AL,Heart Failure,baseline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086418,0.086418,0.086418
1,Alabama,AL,Heart Failure,lifestyle_100,0.000044,0.000044,0.000044,0.050375,0.050375,0.050375,0.086375,0.086375,0.086375
2,Alabama,AL,Heart Failure,outreach_100,0.000199,0.000199,0.000199,0.230833,0.230833,0.230833,0.086219,0.086219,0.086219
3,Alabama,AL,Heart Failure,polypill_100,0.001382,0.001382,0.001382,1.598969,1.598969,1.598969,0.085037,0.085037,0.085037
4,Alabama,AL,Ischemic Heart Disease,baseline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129946,0.129946,0.129946


### Converting Datasets a Bit for Maps

In [19]:
tran_state_rate_map = pd.DataFrame()
tran_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = tran_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
tran_state_rate_map_2023 = tran_state_rate_map.loc[tran_state_rate_map.year==2023].reset_index()
tran_state_rate_map_2023 = tran_state_rate_map_2023.rename(columns = {'rate_mean':'rate_mean_2023','averted_count_mean':'averted_count_mean_2023','averted_percent_mean':'averted_percent_mean_2023'}).reset_index()
tran_state_rate_map_2040 = tran_state_rate_map.loc[tran_state_rate_map.year==2040].reset_index()
tran_state_rate_map_2040 = tran_state_rate_map_2040.rename(columns = {'rate_mean':'rate_mean_2040','averted_count_mean':'averted_count_mean_2040','averted_percent_mean':'averted_percent_mean_2040'}).reset_index()
tran_state_rate_map = pd.concat([tran_state_rate_map_2023, tran_state_rate_map_2040[['rate_mean_2040','averted_count_mean_2040','averted_percent_mean_2040']]], axis=1)#.reset_index()
#tran_state_rate_map.head()

In [20]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
tran_state_rate_map = tran_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#tran_state_rate_map

In [21]:
daly_state_rate_map = pd.DataFrame()
daly_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = daly_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
daly_state_rate_map_2023 = daly_state_rate_map.loc[daly_state_rate_map.year==2023].reset_index()
daly_state_rate_map_2023 = daly_state_rate_map_2023.rename(columns = {'rate_mean':'rate_mean_2023','averted_count_mean':'averted_count_mean_2023','averted_percent_mean':'averted_percent_mean_2023'})#.reset_index()
daly_state_rate_map_2040 = daly_state_rate_map.loc[daly_state_rate_map.year==2040].reset_index()
daly_state_rate_map_2040 = daly_state_rate_map_2040.rename(columns = {'rate_mean':'rate_mean_2040','averted_count_mean':'averted_count_mean_2040','averted_percent_mean':'averted_percent_mean_2040'}).reset_index()
daly_state_rate_map = pd.concat([daly_state_rate_map_2023, daly_state_rate_map_2040[['rate_mean_2040','averted_count_mean_2040','averted_percent_mean_2040']]], axis=1)#.reset_index()
#daly_state_rate_map#_2023#.head()

In [22]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
daly_state_rate_map = daly_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])

# Make Graphs 


### 2023 vs 2040 Maps for all Outcomes

In [23]:
tran_state_rate_map_baseline = tran_state_rate_map.loc[(tran_state_rate_map.scenario == 'baseline')]
daly_state_rate_map_baseline = daly_state_rate_map.loc[(daly_state_rate_map.scenario == 'baseline')]
#tran_state_rate_map

In [35]:
def comparison_years(data_1, outcome): 
    charts = []
    for tran in data_1.transition_new.unique(): 

        subdata = data_1.loc[data_1.transition_new == tran] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['rate_mean_2023', 'rate_mean_2040']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('rate_mean_2023:Q', title= outcome),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'{tran} {outcome} in 2023'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color='rate_mean_2040:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'{tran} {outcome} in 2040'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        chart = alt.hconcat(chart_1, chart_2)

        charts.append(chart)
    return charts

In [36]:
incidence_charts = comparison_years(tran_state_rate_map_baseline, 'Incidence Rate')
alt.vconcat(*incidence_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

In [37]:
daly_charts = comparison_years(daly_state_rate_map_baseline, 'DALY Rate')
alt.vconcat(*daly_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

### Averted Percent by Scenario Maps

In [54]:
tran_state_rate_map = pd.DataFrame()
tran_state_rate_map[['transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = tran_state_rate_allyear_summ[['transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
tran_state_rate_map_polypill = tran_state_rate_map.loc[tran_state_rate_map.scenario=='polypill_100'].reset_index()
tran_state_rate_map_polypill = tran_state_rate_map_polypill.rename(columns = {'rate_mean':'rate_mean_polypill','averted_count_mean':'averted_count_mean_polypill','averted_percent_mean':'averted_percent_mean_polypill'})
tran_state_rate_map_outreach = tran_state_rate_map.loc[tran_state_rate_map.scenario=='outreach_100'].reset_index()
tran_state_rate_map_outreach = tran_state_rate_map_outreach.rename(columns = {'rate_mean':'rate_mean_outreach','averted_count_mean':'averted_count_mean_outreach','averted_percent_mean':'averted_percent_mean_outreach'})
tran_state_rate_map_lifestyle = tran_state_rate_map.loc[tran_state_rate_map.scenario=='lifestyle_100'].reset_index()
tran_state_rate_map_lifestyle = tran_state_rate_map_lifestyle.rename(columns = {'rate_mean':'rate_mean_lifestyle','averted_count_mean':'averted_count_mean_lifestyle','averted_percent_mean':'averted_percent_mean_lifestyle'}).reset_index()
tran_state_rate_map = pd.concat([tran_state_rate_map_polypill, tran_state_rate_map_outreach[['rate_mean_outreach','averted_count_mean_outreach','averted_percent_mean_outreach']], tran_state_rate_map_lifestyle[['rate_mean_lifestyle','averted_count_mean_lifestyle','averted_percent_mean_lifestyle']]], axis=1)
tran_state_rate_map.head()

,index,transition_new,location_name,abbreviation,scenario,rate_mean_polypill,averted_count_mean_polypill,averted_percent_mean_polypill,rate_mean_outreach,averted_count_mean_outreach,averted_percent_mean_outreach,rate_mean_lifestyle,averted_count_mean_lifestyle,averted_percent_mean_lifestyle
0,3,Heart Failure,Alabama,AL,polypill_100,0.015741,0.000546,3.352591,0.016220,0.000067,0.411617,0.016200,0.000088,0.538835
1,7,Ischemic Stroke,Alabama,AL,polypill_100,0.001593,0.000144,8.275258,0.001713,0.000023,1.316052,0.001723,0.000013,0.755602
2,11,Myocardial Infarction,Alabama,AL,polypill_100,0.004586,0.000418,8.354956,0.004932,0.000073,1.451490,0.004950,0.000055,1.090159
3,15,Heart Failure,Alaska,AK,polypill_100,0.011845,0.000461,3.748894,0.012239,0.000068,0.550709,0.012257,0.000050,0.402315
4,19,Ischemic Stroke,Alaska,AK,polypill_100,0.001124,0.000109,8.879721,0.001221,0.000012,0.977364,0.001225,0.000008,0.658173


In [55]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
tran_state_rate_map = tran_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#tran_state_rate_map

In [74]:
daly_state_rate_map = pd.DataFrame()
daly_state_rate_map[['transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = daly_state_allyear_rate_summ[['transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
daly_state_rate_map_polypill = daly_state_rate_map.loc[daly_state_rate_map.scenario=='polypill_100'].reset_index()
daly_state_rate_map_polypill = daly_state_rate_map_polypill.rename(columns = {'rate_mean':'rate_mean_polypill','averted_count_mean':'averted_count_mean_polypill','averted_percent_mean':'averted_percent_mean_polypill'})
daly_state_rate_map_outreach = daly_state_rate_map.loc[daly_state_rate_map.scenario=='outreach_100'].reset_index()
daly_state_rate_map_outreach = daly_state_rate_map_outreach.rename(columns = {'rate_mean':'rate_mean_outreach','averted_count_mean':'averted_count_mean_outreach','averted_percent_mean':'averted_percent_mean_outreach'})
daly_state_rate_map_lifestyle = daly_state_rate_map.loc[daly_state_rate_map.scenario=='lifestyle_100'].reset_index()
daly_state_rate_map_lifestyle = daly_state_rate_map_lifestyle.rename(columns = {'rate_mean':'rate_mean_lifestyle','averted_count_mean':'averted_count_mean_lifestyle','averted_percent_mean':'averted_percent_mean_lifestyle'}).reset_index()
daly_state_rate_map = pd.concat([daly_state_rate_map_polypill, daly_state_rate_map_outreach[['rate_mean_outreach','averted_count_mean_outreach','averted_percent_mean_outreach']], daly_state_rate_map_lifestyle[['rate_mean_lifestyle','averted_count_mean_lifestyle','averted_percent_mean_lifestyle']]], axis=1)
daly_state_rate_map

,index,transition_new,location_name,abbreviation,scenario,rate_mean_polypill,averted_count_mean_polypill,averted_percent_mean_polypill,rate_mean_outreach,averted_count_mean_outreach,averted_percent_mean_outreach,rate_mean_lifestyle,averted_count_mean_lifestyle,averted_percent_mean_lifestyle
0,3,Heart Failure,Alabama,AL,polypill_100,0.085037,0.001382,1.598969,0.086219,0.000199,0.230833,0.086375,0.000044,0.050375
1,7,Ischemic Heart Disease,Alabama,AL,polypill_100,0.125684,0.004262,3.279922,0.129078,0.000868,0.667760,0.129144,0.000802,0.617260
2,11,Ischemic Stroke,Alabama,AL,polypill_100,0.013572,0.000363,2.601813,0.013831,0.000104,0.743742,0.013864,0.000071,0.510922
3,15,Heart Failure,Alaska,AK,polypill_100,0.066042,0.000601,0.901249,0.066449,0.000194,0.291419,0.066526,0.000117,0.176058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,587,Ischemic Stroke,Wisconsin,WI,polypill_100,0.008709,0.000213,2.392480,0.008896,0.000027,0.306295,0.008912,0.000010,0.117632
147,591,Heart Failure,Wyoming,WY,polypill_100,0.068471,0.000288,0.418941,0.068588,0.000171,0.249307,0.068165,0.000594,0.864176
148,595,Ischemic Heart Disease,Wyoming,WY,polypill_100,0.114250,0.003983,3.368603,0.117727,0.000505,0.427296,0.117275,0.000957,0.809788
149,599,Ischemic Stroke,Wyoming,WY,polypill_100,0.010130,0.000160,1.554598,0.010229,0.000061,0.589747,0.010273,0.000017,0.168917


In [70]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
daly_state_rate_map = daly_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#daly_state_rate_map

In [71]:
pop.location_name.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [32]:
def interventions_maps(data_1, outcome): 
    charts = []
    for tran in data_1.transition_new.unique(): 

        subdata = data_1.loc[data_1.transition_new == 'Heart Failure'] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['averted_percent_mean_polypill', 'averted_percent_mean_outreach']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('averted_percent_mean_polypill:Q', title= 'Percent Averted'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} {outcome} Averted with Polypill'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color='averted_percent_mean_outreach:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} {outcome} Averted with Outreach'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        background_3 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_3 = alt.Chart(states).mark_geoshape().encode(
            color='averted_percent_mean_lifestyle:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} Incidence Averted with Lifestyle'
        ).project(
            type='albersUsa'
        )

        chart_3 = alt.layer(background_3, overlay_3)
        chart = alt.hconcat(chart_1, chart_2, chart_3)

        charts.append(chart)
    return charts

In [58]:
incidence_averted_charts = interventions_maps(tran_state_rate_map, 'Incidence')
alt.vconcat(*incidence_averted_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

In [72]:
daly_averted_charts = interventions_maps(daly_state_rate_map, 'DALYs')
alt.vconcat(*daly_averted_charts).resolve_scale(color="independent")

alt.VConcatChart(...)